## 📚 IMPORTAÇÃO DE BIBLIOTECAS

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from openai import OpenAI
from dotenv import load_dotenv

## 🔐 CARREGAR VARIÁVEIS DE AMBIENTE

In [2]:
load_dotenv()

True


## 🤖 CONFIGURAÇÃO OPENAI

In [3]:
client = OpenAI(
    organization='org-IsYPmtrkCS9KEDskoYDCzwa1',
    project='proj_62XWXAEEerK598jeu0YOeuMp',
)

## 🧩 CONEXÃO COM O DATA WAREHOUSE (SNOWFLAKE)

In [4]:
def connect_dw():
    return create_engine(
        f"snowflake://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}?warehouse={os.getenv('DB_WAREHOUSE')}"
    )

In [5]:
def executar_query(query):
    conn = connect_dw()
    with conn.connect() as sql:
        try:
            # Executando a consulta no DW
            result = sql.execute(text(query))

            # Pegando os nomes das colunas automaticamente
            columns = result.keys()

            # Buscando os dados
            data = result.fetchall()

            # Se a consulta retornou dados, retorna o DataFrame
            if data:
                df = pd.DataFrame(data, columns=columns)
                return df
            else:
                return pd.DataFrame()  # Retorna um DataFrame vazio caso não haja dados
        
        except Exception as e:
            # Se houver erro ao executar a query, ele será propagado
            raise Exception(f"Erro ao executar a query: {e}")

## 🔁 PEGAR O ID DO ASSISTENTE CRIADO

In [6]:
my_assistants = client.beta.assistants.list(order="desc", limit=20)
most_recent_assistant_id = my_assistants.data[0].id
print(f"ID do assistente mais recente: {most_recent_assistant_id}")

ID do assistente mais recente: asst_LKdSDm5rPMi3YykJuzi7U6GE


## 🧠 ENVIA A PERGUNTA PARA O ASSISTENTE

In [ ]:
# Criação do assistente personalizado
assistant = client.beta.assistants.create(
    name="Analista de Dados da Feng",
    instructions="""
    Você é um assistente de dados da Feng. Responda sempre em português e apenas como assistente.
Sua função é gerar queries SQL de uma linha (sem quebra) para serem executadas via Python no Snowflake.
Sempre use o schema nas tabelas, por exemplo: BI_DIM.DIM_ASSINANTE.
Nunca adicione explicações, apenas retorne a query pura.

Sobre a tabela BI_DIM.DIM_ASSINANTE:
Contém todas as pessoas que já tiveram contrato

SK_ASSINANTE: chave primária
idpessoa: id no sistema
sk_lead: id na tabela dim_lead
chave_sales_force: id no Salesforce
PROGRAMA: programa da assinatura (ex: SAO_PAULO, FLAMENGO, etc.)
estado_ativacao: status como SÓCIO, INATIVO, LEAD etc
tipo_programa: Esportes ou Multa
sexo: M, F, I ou Null
tipo_pessoa: título ao aderir ao plano
Flags: flag_aceita_newsletter, flag_socio_club, flag_cccredit (valores S ou N)
idpessoa_responsavel: se tem responsável
tenure: meses ativos seguidos
total_meses_ativos: total de meses ativos até sair
idnivel: referência à dim_nivel
""",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o-mini",
)

# Guardando o ID do assistente criado
custom_assistant_id = assistant.id
print(f"Assistente personalizado criado com ID: {custom_assistant_id}")

Assistente personalizado criado com ID: asst_QyjCVxiu7ARYqpZ25h0LbQ6u


In [8]:
def realizar_consulta_assistente(prompt_usuario):
    thread = client.beta.threads.create()
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt_usuario
    )

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=custom_assistant_id,  # usa o assistente recém-criado
    )

    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )

    messages = client.beta.threads.messages.list(thread_id=thread.id)
    resposta = messages.data[0].content[0].text.value

    return resposta

## 🗣️ CAPTURA A PERGUNTA DO USUÁRIO E FAZ A CONSULTA NO SNOWFLAKE

In [9]:
def loop_consultas():
    while True:
        # Passo 1: Solicita ao usuário a consulta
        prompt_usuario = input("\n🗣️ O que você quer saber?\nEx: 'Quantos sócios temos ?'\n\n> ")

        if prompt_usuario.lower() == "sair":
            print("\n🚪 Saindo do sistema de consultas.")
            break  # Encerra o loop se o usuário digitar 'sair'
        
        # Passo 2: Realiza a consulta com o assistente
        print(f"\n💬 Sua pergunta foi: {prompt_usuario}")
        print("\n⚙️ Enviando consulta ao assistente...")
        resposta = realizar_consulta_assistente(prompt_usuario)
        print(f"\n📜 Query gerada pelo assistente:\n{resposta}")

        # Passo 3: Executa a query recebida
        try:
            print("\n⚙️ Executando query no DW...")
            df_resultado = executar_query(resposta)
            print("\n✅ Resultado:")
            display(df_resultado)
        except Exception as e:
            print(f"\n❌ Erro ao executar a query:\n{e}")

In [10]:
loop_consultas()


💬 Sua pergunta foi: 

⚙️ Enviando consulta ao assistente...


BadRequestError: Error code: 400 - {'error': {'message': 'Message content must be non-empty.', 'type': 'invalid_request_error', 'param': 'content', 'code': None}}